# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso
from tensorflow.keras.models import load_model

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.32
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Function for calculating BER

In [5]:
# Funzione per calcolare il BER
def calculate_ber(model, snr_test_data_path, max_samples=500000):
    ber_values = []
    snr_points = list(range(0, 21))

    for snr_db in tqdm(snr_points, desc="Calculating BER"):
        test_file = os.path.join(snr_test_data_path, f"test_{snr_db}_SNR.npz")
        if not os.path.exists(test_file):
            print(f"File non trovato: {test_file}. Salto questo SNR.")
            ber_values.append(np.nan)
            continue

        data = np.load(test_file)
        X_batch, y_true_batch = data['X_test'], data['y_test']
        symbols_in_batch = X_batch.shape[0]

        if symbols_in_batch == 0:
            ber_values.append(np.nan)
            continue

         # --- Estrazione casuale di un sottoinsieme ---
        if len(X_batch) > max_samples:
            print("entrato nell'if di riduzione campioni per ridurre tempi.")
            idx = np.random.choice(len(X_batch), max_samples, replace=False)
            X_batch = X_batch[idx]
            y_true_batch = y_true_batch[idx]

        predictions = model.predict(X_batch, batch_size=10000, verbose=0)
        predicted_labels = np.argmax(predictions, axis=1)
        errors = np.sum(predicted_labels != y_true_batch)
        ber = errors / symbols_in_batch
        ber_values.append(ber)
        print(f"SNR: {snr_db} dB, BER: {ber:.6f}")

    return np.array(ber_values), snr_points

# Funzione di plotting statica (non più usata)

In [6]:
# Funzione di plotting

def plot_ber(snr_points, ber_curves_dict, channel_type, title_suffix="", filename="ber_plot.png", snr_x_lims=(0, 20)):
    """
    Plotta BER vs SNR per una serie di modelli. Ogni modello ha una curva nel dizionario.

    Args:
        snr_points (list or np.array): Valori di SNR (in dB).
        ber_curves_dict (dict): {nome_modello: lista_ber}
        channel_type (str): 'AWGN', 'Rayleigh', ecc.
        title_suffix (str): Testo da aggiungere al titolo del grafico.
        filename (str): Nome del file immagine da salvare.
        snr_x_lims (tuple): Range dell'asse X (min, max).
    """
    plt.figure(figsize=(10, 6))

    for label, ber_data_orig in ber_curves_dict.items():
        ber_data = np.array(ber_data_orig)

        # Adatta la lunghezza dei dati
        if len(ber_data) < len(snr_points):
            temp_ber_data = np.full(len(snr_points), np.nan)
            temp_ber_data[:len(ber_data)] = ber_data
            ber_data = temp_ber_data
        elif len(ber_data) > len(snr_points):
            ber_data = ber_data[:len(snr_points)]

        # Applica la logica "tocca l'asse X"
        ber_to_plot = ber_data.copy()
        for i, ber_val in enumerate(ber_data):
            if not np.isnan(ber_val) and ber_val <= 1e-15:
                ber_to_plot[i] = 1e-5
                ber_to_plot[i+1:] = np.nan
                break

        # Marker diverso in base al range di addestramento
        marker = 'x' if "11-15" in label else 'o'

        # Plot della curva BER
        plt.semilogy(snr_points, ber_to_plot, marker=marker, linestyle='-', label=label)

    plt.title(f'Bit Error Rate (BER) vs. SNR for {channel_type} Channel {title_suffix}')
    plt.xlabel('SNR (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.legend()
    plt.ylim(1e-5, 1)

    # Limiti e ticks asse X
    min_snr, max_snr = snr_x_lims
    plt.xticks(np.arange(int(min_snr), int(max_snr) + 1, 2))
    plt.xlim(min_snr, max_snr)

    plt.savefig(filename)
    plt.show()
    print(f"Grafico salvato come {filename}")

# Calculate the BER

In [7]:
# === Impostazioni di base ===
cnn_model_folder = "/content/drive/MyDrive/GitHub/Rayleigh/trained_models/cnn"
snr_test_data_path = "/content/drive/MyDrive/GitHub/Rayleigh/dataset/test"
results_folder = "/content/drive/MyDrive/GitHub/Rayleigh/results"

os.makedirs(results_folder, exist_ok=True)  # Crea la cartella dei risultati se non esiste

ber_results = {}
snr_points = None  # Verrà popolato durante il primo calcolo

for model_file in os.listdir(cnn_model_folder):
    if model_file.endswith(".h5"):
        model_name = model_file.replace(".h5", "")
        model_path = os.path.join(cnn_model_folder, model_file)
        result_file = os.path.join(results_folder, f"{model_name}_ber.npy")

        # Se esistono già i risultati, caricali
        if os.path.exists(result_file):
            print(f"Risultati già presenti per {model_name}, caricamento da file...")
            ber_values = np.load(result_file)
        else:
            print(f"\nCaricamento modello: {model_name}")
            model = load_model(model_path)
            ber_values, snr_points = calculate_ber(model, snr_test_data_path)
            np.save(result_file, ber_values)  # Salva risultati
            print(f"BER salvato in: {result_file}")

        ber_results[model_name] = ber_values
        print(f"BER per {model_name}: {ber_values}")

# # === Plot finale ===
# plot_ber(
#     snr_points,
#     ber_results,
#     channel_type="Rayleigh",
#     title_suffix="CNN Evaluation",
#     filename=os.path.join(results_folder, "ber_cnn_all_models.png")
# )


Risultati già presenti per CNN_3-ksize_128-filters_rayleigh_snr_0-20, caricamento da file...
BER per CNN_3-ksize_128-filters_rayleigh_snr_0-20: [0.356308 0.325672 0.289784 0.255246 0.21824  0.182904 0.151352 0.121442
 0.095864 0.074302 0.055674 0.039272 0.027676 0.021296 0.020298 0.022638
 0.025286 0.027734 0.030458 0.033038 0.035516]
Risultati già presenti per CNN_5-ksize_128-filters_rayleigh_snr_0-20, caricamento da file...
BER per CNN_5-ksize_128-filters_rayleigh_snr_0-20: [0.36621  0.322428 0.2753   0.228492 0.1827   0.144118 0.112938 0.087338
 0.067338 0.05184  0.03938  0.028686 0.021156 0.016606 0.014832 0.015324
 0.01631  0.017206 0.01874  0.020018 0.021658]
Risultati già presenti per CNN_7-ksize_128-filters_rayleigh_snr_0-20, caricamento da file...
BER per CNN_7-ksize_128-filters_rayleigh_snr_0-20: [0.334202 0.28589  0.238884 0.194576 0.153802 0.121746 0.09593  0.075804
 0.060314 0.048546 0.03897  0.029732 0.021906 0.016058 0.012478 0.01168
 0.012078 0.0132   0.014728 0.016006 

TypeError: object of type 'NoneType' has no len()

<Figure size 1000x600 with 0 Axes>

# Print the single interactive CNN setup.

In [8]:
import os
import numpy as np
import plotly.graph_objs as go

results_dir = "/content/drive/MyDrive/GitHub/Rayleigh/results"
ber_files = [f for f in os.listdir(results_dir) if f.endswith("_ber.npy")]

# Esempio: SNR reali usati (modifica se necessario)
snr_points = np.array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20])

ber_data = {}
for f in ber_files:
    model_name = f.replace("_ber.npy", "")
    path = os.path.join(results_dir, f)
    ber = np.load(path)
    valid_ber = ber[~np.isnan(ber)]
    if len(valid_ber) > 0:
        ber_data[model_name] = {
            "ber": ber,
            "mean": np.mean(valid_ber)
        }

# Trova miglior modello globale
best_model = min(ber_data.items(), key=lambda x: x[1]["mean"])
best_model_name = best_model[0]

# Crea grafico interattivo
fig = go.Figure()

for model_name, info in ber_data.items():
    visibility = True if model_name == best_model_name else 'legendonly'
    fig.add_trace(go.Scatter(
        x=snr_points,
        y=info["ber"],
        mode='lines+markers',
        name=model_name,
        visible=visibility,
        line=dict(width=3 if model_name == best_model_name else 1.5)
    ))

fig.update_layout(
    title="📉 Curve BER Interattive per Modelli CNN",
    xaxis_title="SNR (dB)",
    yaxis_title="Bit Error Rate (BER)",
    yaxis_type="log",
    hovermode="x unified",
    legend_title="Modelli",
    template="plotly_white"
)

fig.show()
